In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [44]:
compas_data = pd.read_csv('data/propublica_data_for_fairml.csv')
x = compas_data.iloc[:, [1,2,3,4,5,6,7,8,9,10,11]].values
y = compas_data.iloc[:, 0].values

In [45]:
compas_data = [(x,y)]

In [46]:
classifiers = [KNeighborsClassifier(3), SVC(kernel="linear", C=0.025), SVC(gamma=2, C=1), DecisionTreeClassifier(max_depth=5), 
               RandomForestClassifier(max_depth=5, n_estimators=10), MLPClassifier(alpha=1), AdaBoostClassifier(), GaussianNB(),
               QuadraticDiscriminantAnalysis(),LogisticRegression()]

In [47]:
for ds_cnt, ds in enumerate(compas_data):
    x, y = ds
    x = StandardScaler().fit_transform(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=42)

In [48]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", 
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA","LogisticRegression"]

In [49]:
for name, clf in zip(names, classifiers):
    clf.fit(x_train, y_train)
    score = clf.score(x_test, y_test)
    print(name,score)

Nearest Neighbors 0.6123110151187905
Linear SVM 0.652267818574514
RBF SVM 0.6641468682505399
Decision Tree 0.6760259179265659
Random Forest 0.6711663066954644
Neural Net 0.673866090712743
AdaBoost 0.66792656587473
Naive Bayes 0.6241900647948164
QDA 0.6452483801295896
LogisticRegression 0.6652267818574514


In [50]:
from scipy.stats import randint as sp_randint
search_parameters_space_random = {"max_depth": [3, None], "max_features": sp_randint(1, 9),
                                  "min_samples_leaf": sp_randint(1, 9), "criterion": ["gini","entropy"]}

In [57]:
random_search = RandomizedSearchCV(estimator=DecisionTreeClassifier(), param_distributions=search_parameters_space_random,
                                   scoring="roc_auc", n_jobs=-1, n_iter=50)

In [64]:
random_search.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(), n_iter=50, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, None],
                                        'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002C2E884BE80>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002C2E98865E0>},
                   scoring='roc_auc')

In [66]:
# Best Score
random_search.best_score_

0.719840807891299

In [67]:
# Best Estimator
random_search.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=3, max_features=8,
                       min_samples_leaf=4)

In [68]:
# Best Param
random_search.best_params_

{'criterion': 'entropy',
 'max_depth': 3,
 'max_features': 8,
 'min_samples_leaf': 4}

In [69]:
y_pred = random_search.predict(x_test)

# Print Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.78      0.73      1020
           1       0.67      0.54      0.60       832

    accuracy                           0.67      1852
   macro avg       0.67      0.66      0.66      1852
weighted avg       0.67      0.67      0.67      1852

